# การทำงานแบบ Human-in-the-Loop ด้วย Microsoft Agent Framework

## 🎯 วัตถุประสงค์การเรียนรู้

ในโน้ตบุ๊กนี้ คุณจะได้เรียนรู้วิธีการสร้าง **การทำงานแบบ Human-in-the-Loop** โดยใช้ `RequestInfoExecutor` ของ Microsoft Agent Framework รูปแบบที่ทรงพลังนี้ช่วยให้คุณหยุดการทำงานของ AI เพื่อรวบรวมข้อมูลจากมนุษย์ ทำให้ตัวแทน AI มีความโต้ตอบและให้มนุษย์ควบคุมการตัดสินใจที่สำคัญ

## 🔄 Human-in-the-Loop คืออะไร?

**Human-in-the-loop (HITL)** เป็นรูปแบบการออกแบบที่ตัวแทน AI หยุดการทำงานเพื่อขอข้อมูลจากมนุษย์ก่อนดำเนินการต่อ ซึ่งมีความสำคัญสำหรับ:

- ✅ **การตัดสินใจที่สำคัญ** - ขอการอนุมัติจากมนุษย์ก่อนดำเนินการที่สำคัญ
- ✅ **สถานการณ์ที่ไม่ชัดเจน** - ให้มนุษย์ช่วยชี้แจงเมื่อ AI ไม่แน่ใจ
- ✅ **ความชอบของผู้ใช้** - ขอให้ผู้ใช้เลือกจากตัวเลือกหลายตัว
- ✅ **การปฏิบัติตามกฎระเบียบและความปลอดภัย** - ให้มนุษย์ตรวจสอบการดำเนินการที่มีการควบคุม
- ✅ **ประสบการณ์แบบโต้ตอบ** - สร้างตัวแทนสนทนาที่ตอบสนองต่อข้อมูลจากผู้ใช้

## 🏗️ วิธีการทำงานใน Microsoft Agent Framework

เฟรมเวิร์กนี้มีองค์ประกอบสำคัญสามอย่างสำหรับ HITL:

1. **`RequestInfoExecutor`** - ตัวดำเนินการพิเศษที่หยุดการทำงานและส่ง `RequestInfoEvent`
2. **`RequestInfoMessage`** - คลาสพื้นฐานสำหรับข้อมูลคำขอที่ส่งไปยังมนุษย์
3. **`RequestResponse`** - เชื่อมโยงการตอบกลับของมนุษย์กับคำขอเดิมโดยใช้ `request_id`

**รูปแบบการทำงาน:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 ตัวอย่างของเรา: การจองโรงแรมพร้อมการยืนยันจากผู้ใช้

เราจะสร้างการทำงานแบบมีเงื่อนไขโดยเพิ่มการยืนยันจากมนุษย์ **ก่อน** แนะนำจุดหมายปลายทางทางเลือก:

1. ผู้ใช้ขอจุดหมายปลายทาง (เช่น "ปารีส")
2. `availability_agent` ตรวจสอบว่ามีห้องว่างหรือไม่
3. **หากไม่มีห้องว่าง** → `confirmation_agent` ถามว่า "คุณต้องการดูตัวเลือกอื่นหรือไม่?"
4. การทำงาน **หยุดชั่วคราว** โดยใช้ `RequestInfoExecutor`
5. **มนุษย์ตอบกลับ** "ใช่" หรือ "ไม่" ผ่านการป้อนข้อมูลในคอนโซล
6. `decision_manager` ตัดสินใจตามคำตอบ:
   - **ใช่** → แสดงจุดหมายปลายทางทางเลือก
   - **ไม่** → ยกเลิกคำขอจอง
7. แสดงผลลัพธ์สุดท้าย

ตัวอย่างนี้แสดงให้เห็นถึงวิธีการให้ผู้ใช้ควบคุมคำแนะนำของตัวแทน!

---

มาเริ่มกันเลย! 🚀


## ขั้นตอนที่ 1: นำเข้าไลบรารีที่จำเป็น

เรานำเข้าส่วนประกอบมาตรฐานของ Agent Framework รวมถึง **คลาสเฉพาะสำหรับการมีมนุษย์ในกระบวนการ**:
- `RequestInfoExecutor` - ตัวดำเนินการที่หยุดการทำงานของเวิร์กโฟลว์เพื่อรอการป้อนข้อมูลจากมนุษย์
- `RequestInfoEvent` - เหตุการณ์ที่ถูกส่งออกเมื่อมีการร้องขอข้อมูลจากมนุษย์
- `RequestInfoMessage` - คลาสพื้นฐานสำหรับข้อมูลคำร้องที่มีประเภท
- `RequestResponse` - เชื่อมโยงการตอบกลับของมนุษย์กับคำร้องขอ
- `WorkflowOutputEvent` - เหตุการณ์สำหรับตรวจจับผลลัพธ์ของเวิร์กโฟลว์


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## ขั้นตอนที่ 2: กำหนดโมเดล Pydantic สำหรับผลลัพธ์ที่มีโครงสร้าง

โมเดลเหล่านี้กำหนด **schema** ที่ตัวแทนจะส่งคืน เรารักษาโมเดลทั้งหมดจากกระบวนการทำงานแบบมีเงื่อนไขและเพิ่ม:

**ใหม่สำหรับ Human-in-the-Loop:**
- `HumanFeedbackRequest` - คลาสย่อยของ `RequestInfoMessage` ที่กำหนดข้อมูลคำขอที่ส่งไปยังมนุษย์
  - ประกอบด้วย `prompt` (คำถามที่จะถาม) และ `destination` (บริบทเกี่ยวกับเมืองที่ไม่สามารถเข้าถึงได้)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## ขั้นตอนที่ 3: สร้างเครื่องมือจองโรงแรม

เครื่องมือเดียวกันจากกระบวนการทำงานแบบมีเงื่อนไข - ตรวจสอบว่ามีห้องว่างในจุดหมายปลายทางหรือไม่


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ขั้นตอนที่ 4: กำหนดฟังก์ชันเงื่อนไขสำหรับการกำหนดเส้นทาง

เราต้องการ **ฟังก์ชันเงื่อนไขสี่ตัว** สำหรับเวิร์กโฟลว์ที่มีมนุษย์เข้ามาเกี่ยวข้อง:

**จากเวิร์กโฟลว์แบบมีเงื่อนไข:**
1. `has_availability_condition` - กำหนดเส้นทางเมื่อมีโรงแรมว่าง
2. `no_availability_condition` - กำหนดเส้นทางเมื่อไม่มีโรงแรมว่าง

**ใหม่สำหรับเวิร์กโฟลว์ที่มีมนุษย์เข้ามาเกี่ยวข้อง:**
3. `user_wants_alternatives_condition` - กำหนดเส้นทางเมื่อผู้ใช้ตอบว่า "ใช่" สำหรับตัวเลือกอื่น
4. `user_declines_alternatives_condition` - กำหนดเส้นทางเมื่อผู้ใช้ตอบว่า "ไม่" สำหรับตัวเลือกอื่น


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## ขั้นตอนที่ 5: สร้าง Decision Manager Executor

นี่คือ **หัวใจสำคัญของรูปแบบ human-in-the-loop**! `DecisionManager` เป็น `Executor` แบบกำหนดเองที่:

1. **รับความคิดเห็นจากมนุษย์** ผ่าน `RequestResponse` objects
2. **ประมวลผลการตัดสินใจของผู้ใช้** (ใช่/ไม่ใช่)
3. **กำหนดเส้นทางของ workflow** โดยการส่งข้อความไปยังตัวแทนที่เหมาะสม

คุณสมบัติสำคัญ:
- ใช้ `@handler` decorator เพื่อเปิดเผยเมธอดเป็นขั้นตอนใน workflow
- รับ `RequestResponse[HumanFeedbackRequest, str]` ซึ่งประกอบด้วยคำขอเดิมและคำตอบของผู้ใช้
- ส่งข้อความง่าย ๆ "ใช่" หรือ "ไม่ใช่" ที่กระตุ้นฟังก์ชันเงื่อนไขของเรา


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## ขั้นตอนที่ 6: สร้างตัวดำเนินการแสดงผลแบบกำหนดเอง

ตัวดำเนินการแสดงผลเดียวกันจาก workflow แบบมีเงื่อนไข - ให้ผลลัพธ์สุดท้ายเป็นผลลัพธ์ของ workflow


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## ขั้นตอนที่ 7: โหลดตัวแปรสภาพแวดล้อม

ตั้งค่าคอนฟิก LLM client (GitHub Models, Azure OpenAI หรือ OpenAI)


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## ขั้นตอนที่ 8: สร้างตัวแทน AI และตัวดำเนินการ

เราสร้าง **องค์ประกอบของเวิร์กโฟลว์หกส่วน**:

**ตัวแทน (ห่อด้วย AgentExecutor):**
1. **availability_agent** - ตรวจสอบความพร้อมของโรงแรมโดยใช้เครื่องมือ
2. **confirmation_agent** - 🆕 เตรียมคำขอการยืนยันจากมนุษย์
3. **alternative_agent** - แนะนำเมืองทางเลือก (เมื่อผู้ใช้ตอบว่าใช่)
4. **booking_agent** - สนับสนุนการจอง (เมื่อมีห้องว่าง)
5. **cancellation_agent** - 🆕 จัดการข้อความยกเลิก (เมื่อผู้ใช้ตอบว่าไม่)

**ตัวดำเนินการพิเศษ:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` ที่หยุดเวิร์กโฟลว์เพื่อรอข้อมูลจากมนุษย์
7. **decision_manager** - 🆕 ตัวดำเนินการแบบกำหนดเองที่จัดการเส้นทางตามการตอบสนองของมนุษย์ (ได้กำหนดไว้แล้วด้านบน)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## ขั้นตอนที่ 9: สร้าง Workflow พร้อม Human-in-the-Loop

ตอนนี้เราจะสร้างกราฟ workflow โดยมี **การกำหนดเส้นทางตามเงื่อนไข** รวมถึงเส้นทางที่มี human-in-the-loop:

**โครงสร้าง Workflow:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**เส้นทางสำคัญ:**
- `availability_agent → confirmation_agent` (เมื่อไม่มีห้องว่าง)
- `confirmation_agent → prepare_human_request` (เปลี่ยนประเภท)
- `prepare_human_request → request_info_executor` (หยุดรอมนุษย์)
- `request_info_executor → decision_manager` (เสมอ - ให้ RequestResponse)
- `decision_manager → alternative_agent` (เมื่อผู้ใช้ตอบว่า "ใช่")
- `decision_manager → cancellation_agent` (เมื่อผู้ใช้ตอบว่า "ไม่")
- `availability_agent → booking_agent` (เมื่อมีห้องว่าง)
- ทุกเส้นทางสิ้นสุดที่ `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## ขั้นตอนที่ 10: ทดสอบกรณีที่ 1 - เมืองที่ไม่มีห้องว่าง (ปารีสพร้อมการยืนยันจากมนุษย์)

การทดสอบนี้แสดงให้เห็นถึง **วงจรการทำงานที่มีมนุษย์เข้ามาเกี่ยวข้องอย่างเต็มรูปแบบ**:

1. ขอจองโรงแรมในปารีส
2. availability_agent ตรวจสอบ → ไม่มีห้องว่าง
3. confirmation_agent สร้างคำถามสำหรับมนุษย์
4. request_info_executor **หยุดการทำงานชั่วคราว** และส่ง `RequestInfoEvent`
5. **แอปพลิเคชันตรวจจับเหตุการณ์และแจ้งผู้ใช้ผ่านคอนโซล**
6. ผู้ใช้พิมพ์ "yes" หรือ "no"
7. แอปพลิเคชันส่งคำตอบผ่าน `send_responses_streaming()`
8. decision_manager ตัดสินใจตามคำตอบ
9. แสดงผลลัพธ์สุดท้าย

**รูปแบบสำคัญ:**
- ใช้ `workflow.run_stream()` สำหรับการทำงานรอบแรก
- ใช้ `workflow.send_responses_streaming(pending_responses)` สำหรับการทำงานรอบถัดไป
- ฟัง `RequestInfoEvent` เพื่อตรวจจับเมื่อจำเป็นต้องมีการป้อนข้อมูลจากมนุษย์
- ฟัง `WorkflowOutputEvent` เพื่อจับผลลัพธ์สุดท้าย


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## ขั้นตอนที่ 11: ทดสอบกรณีที่ 2 - เมืองที่มีห้องว่าง (สตอกโฮล์ม - ไม่ต้องการการป้อนข้อมูลจากมนุษย์)

การทดสอบนี้แสดงให้เห็นถึง **เส้นทางตรง** เมื่อมีห้องว่าง:

1. ขอจองโรงแรมในสตอกโฮล์ม  
2. availability_agent ตรวจสอบ → มีห้องว่าง ✅  
3. booking_agent แนะนำการจอง  
4. display_result แสดงการยืนยัน  
5. **ไม่ต้องการการป้อนข้อมูลจากมนุษย์!**

กระบวนการทำงานจะข้ามเส้นทางที่ต้องมีมนุษย์เข้ามาเกี่ยวข้องโดยสิ้นเชิงเมื่อมีห้องว่าง.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## ประเด็นสำคัญและแนวปฏิบัติที่ดีที่สุดสำหรับ Human-in-the-Loop

### ✅ สิ่งที่คุณได้เรียนรู้:

#### 1. **RequestInfoExecutor Pattern**
รูปแบบ human-in-the-loop ใน Microsoft Agent Framework ใช้ส่วนประกอบสำคัญสามอย่าง:
- `RequestInfoExecutor` - หยุดการทำงานของ workflow และส่งเหตุการณ์ออกไป
- `RequestInfoMessage` - คลาสพื้นฐานสำหรับ payload ของคำขอที่มีประเภท (ควร subclass!)
- `RequestResponse` - เชื่อมโยงการตอบกลับของมนุษย์กับคำขอเดิม

**ความเข้าใจที่สำคัญ:**
- `RequestInfoExecutor` ไม่ได้รวบรวมข้อมูลเอง - มันแค่หยุด workflow
- โค้ดแอปพลิเคชันของคุณต้องฟัง `RequestInfoEvent` และรวบรวมข้อมูล
- คุณต้องเรียก `send_responses_streaming()` พร้อม dict ที่จับคู่ `request_id` กับคำตอบของผู้ใช้

#### 2. **Streaming Execution Pattern**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Event-Driven Architecture**
ฟังเหตุการณ์เฉพาะเพื่อควบคุม workflow:
- `RequestInfoEvent` - ต้องการข้อมูลจากมนุษย์ (workflow หยุดชั่วคราว)
- `WorkflowOutputEvent` - ผลลัพธ์สุดท้ายพร้อมใช้งาน (workflow เสร็จสิ้น)
- `WorkflowStatusEvent` - การเปลี่ยนแปลงสถานะ (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, ฯลฯ)

#### 4. **Custom Executors with @handler**
`DecisionManager` แสดงให้เห็นวิธีสร้าง executors ที่:
- ใช้ `@handler` decorator เพื่อเปิดเผยเมธอดเป็นขั้นตอนของ workflow
- รับข้อความที่มีประเภท (เช่น `RequestResponse[HumanFeedbackRequest, str]`)
- ส่งข้อความไปยัง executors อื่นเพื่อกำหนดเส้นทาง workflow
- เข้าถึง context ผ่าน `WorkflowContext`

#### 5. **Conditional Routing with Human Decisions**
คุณสามารถสร้างฟังก์ชันเงื่อนไขที่ประเมินการตอบกลับของมนุษย์:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 การใช้งานในโลกจริง:

1. **Approval Workflows**
   - ขออนุมัติจากผู้จัดการก่อนดำเนินการรายงานค่าใช้จ่าย
   - ต้องการการตรวจสอบจากมนุษย์ก่อนส่งอีเมลอัตโนมัติ
   - ยืนยันธุรกรรมที่มีมูลค่าสูงก่อนดำเนินการ

2. **Content Moderation**
   - ทำเครื่องหมายเนื้อหาที่น่าสงสัยเพื่อให้มนุษย์ตรวจสอบ
   - ขอให้ผู้ดูแลตัดสินใจขั้นสุดท้ายในกรณีที่ไม่ชัดเจน
   - ส่งต่อให้มนุษย์เมื่อ AI มีความมั่นใจต่ำ

3. **Customer Service**
   - ให้ AI จัดการคำถามทั่วไปโดยอัตโนมัติ
   - ส่งต่อปัญหาที่ซับซ้อนให้กับเจ้าหน้าที่มนุษย์
   - ถามลูกค้าว่าต้องการพูดคุยกับมนุษย์หรือไม่

4. **Data Processing**
   - ขอให้มนุษย์แก้ไขข้อมูลที่ไม่ชัดเจน
   - ยืนยันการตีความเอกสารที่ไม่ชัดเจนของ AI
   - ให้ผู้ใช้เลือกจากการตีความที่ถูกต้องหลายแบบ

5. **Safety-Critical Systems**
   - ต้องการการยืนยันจากมนุษย์ก่อนดำเนินการที่ไม่สามารถย้อนกลับได้
   - ขออนุมัติก่อนเข้าถึงข้อมูลที่ละเอียดอ่อน
   - ยืนยันการตัดสินใจในอุตสาหกรรมที่มีการควบคุม (สุขภาพ, การเงิน)

6. **Interactive Agents**
   - สร้างบอทสนทนาที่ถามคำถามติดตามผล
   - สร้างตัวช่วยที่นำทางผู้ใช้ผ่านกระบวนการที่ซับซ้อน
   - ออกแบบตัวแทนที่ร่วมมือกับมนุษย์ทีละขั้นตอน

### 🔄 เปรียบเทียบ: มี vs ไม่มี Human-in-the-Loop

| คุณสมบัติ | Conditional Workflow | Human-in-the-Loop Workflow |
|-----------|---------------------|---------------------------|
| **การดำเนินการ** | `workflow.run()` เดี่ยว | วนลูปด้วย `run_stream()` + `send_responses_streaming()` |
| **การป้อนข้อมูลผู้ใช้** | ไม่มี (อัตโนมัติเต็มรูปแบบ) | การแจ้งเตือนแบบโต้ตอบผ่าน `input()` หรือ UI |
| **ส่วนประกอบ** | Agents + Executors | + RequestInfoExecutor + DecisionManager |
| **เหตุการณ์** | เฉพาะ AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, ฯลฯ |
| **การหยุดชั่วคราว** | ไม่มีการหยุดชั่วคราว | Workflow หยุดชั่วคราวที่ RequestInfoExecutor |
| **การควบคุมโดยมนุษย์** | ไม่มีการควบคุมโดยมนุษย์ | มนุษย์ตัดสินใจสำคัญ |
| **กรณีการใช้งาน** | ระบบอัตโนมัติ | การทำงานร่วมกันและการกำกับดูแล |

### 🚀 รูปแบบขั้นสูง:

#### จุดตัดสินใจของมนุษย์หลายจุด
คุณสามารถมี `RequestInfoExecutor` หลายตัวใน workflow เดียวกัน:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### การจัดการ Timeout
ตั้งค่า timeout สำหรับการตอบกลับของมนุษย์:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### การผสานรวม UI ที่หลากหลาย
แทนที่จะใช้ `input()` ให้ผสานรวมกับ web UI, Slack, Teams ฯลฯ:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Human-in-the-Loop แบบมีเงื่อนไข
ขอข้อมูลจากมนุษย์เฉพาะในสถานการณ์ที่กำหนด:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ แนวปฏิบัติที่ดีที่สุด:

1. **Subclass RequestInfoMessage เสมอ**
   - ให้ความปลอดภัยของประเภทและการตรวจสอบ
   - เปิดใช้งาน context ที่หลากหลายสำหรับการแสดงผล UI
   - ชี้แจงเจตนาของแต่ละประเภทคำขอ

2. **ใช้ข้อความแจ้งที่อธิบายได้**
   - รวม context เกี่ยวกับสิ่งที่คุณถาม
   - อธิบายผลกระทบของแต่ละตัวเลือก
   - ทำให้คำถามง่ายและชัดเจน

3. **จัดการการป้อนข้อมูลที่ไม่คาดคิด**
   - ตรวจสอบการตอบกลับของผู้ใช้
   - ให้ค่าเริ่มต้นสำหรับการป้อนข้อมูลที่ไม่ถูกต้อง
   - ให้ข้อความแสดงข้อผิดพลาดที่ชัดเจน

4. **ติดตาม Request IDs**
   - ใช้การเชื่อมโยงระหว่าง request_id และการตอบกลับ
   - อย่าพยายามจัดการสถานะด้วยตนเอง

5. **ออกแบบให้ไม่บล็อก**
   - อย่าบล็อก threads ขณะรอการป้อนข้อมูล
   - ใช้รูปแบบ async ตลอด
   - รองรับ workflow instances ที่ทำงานพร้อมกัน

### 📚 แนวคิดที่เกี่ยวข้อง:

- **Agent Middleware** - สกัดกั้นการเรียก agent (ในโน้ตบุ๊กก่อนหน้า)
- **Workflow State Management** - บันทึกสถานะ workflow ระหว่างการทำงาน
- **Multi-Agent Collaboration** - รวม human-in-the-loop กับทีม agent
- **Event-Driven Architectures** - สร้างระบบที่ตอบสนองด้วยเหตุการณ์

---

### 🎓 ยินดีด้วย!

คุณได้เชี่ยวชาญ workflow แบบ human-in-the-loop ด้วย Microsoft Agent Framework แล้ว! คุณรู้วิธี:
- ✅ หยุด workflow เพื่อรวบรวมข้อมูลจากมนุษย์
- ✅ ใช้ RequestInfoExecutor และ RequestInfoMessage
- ✅ จัดการการดำเนินการแบบสตรีมด้วยเหตุการณ์
- ✅ สร้าง custom executors ด้วย @handler
- ✅ กำหนดเส้นทาง workflow ตามการตัดสินใจของมนุษย์
- ✅ สร้าง AI agents แบบโต้ตอบที่ร่วมมือกับมนุษย์

**นี่เป็นรูปแบบสำคัญสำหรับการสร้างระบบ AI ที่น่าเชื่อถือและควบคุมได้!** 🚀



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลสำคัญ แนะนำให้ใช้บริการแปลภาษามนุษย์ที่มีความเชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
